# Bayesian MLP vs DropOut MLP

In [1]:
from sklearn.datasets import fetch_mldata
import numpy as np
np.random.seed(0)
import torch
torch.manual_seed(0)
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from mlp import MLP, BayesMLP
from utils import *

In [2]:
mnist = fetch_mldata('MNIST original')
N = 10000

data = np.float32(mnist.data[:]) / 255.
idx = np.random.choice(data.shape[0], N)
data = data[idx]
target = np.int32(mnist.target[idx]).reshape(N, 1)

In [3]:
train_idx, test_idx = train_test_split(np.array(range(N)), test_size=0.05)
train_data, test_data = data[train_idx], data[test_idx]
train_target, test_target = target[train_idx], target[test_idx]

In [4]:
input_size = 784
output_size = 10
M = train_data.shape[0]
learning_rate = 0.001
n_epochs = 100

# Bayesian MLP

In [5]:
train_target_oh = np.float32(preprocessing.OneHotEncoder(sparse=False).fit_transform(train_target))

sigma_prior = float(np.exp(-3))
n_samples = 3

net = BayesMLP(input_size, output_size, sigma_prior)
net.cuda()

log_pw, log_qw, log_likelihood = 0., 0., 0.
batch_size = 100
n_batches = M / float(batch_size)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

n_train_batches = int(train_data.shape[0] / float(batch_size))

In [6]:
def forward_pass_samples(X, y):
    s_log_pw, s_log_qw, s_log_likelihood = 0., 0., 0.
    for _ in xrange(n_samples):
        output = net(X)
        sample_log_pw, sample_log_qw = net.get_lpw_lqw()
        sample_log_likelihood = log_gaussian(y, output, sigma_prior).sum()
        s_log_pw += sample_log_pw
        s_log_qw += sample_log_qw
        s_log_likelihood += sample_log_likelihood
    return s_log_pw/n_samples, s_log_qw/n_samples, s_log_likelihood/n_samples

In [ ]:
for ne in range(n_epochs):
    
    trainlosses = []
    for b in range(n_train_batches):
        
        X = Variable(torch.Tensor(train_data[b * batch_size: (b+1) * batch_size])).cuda()
        y = Variable(torch.Tensor(train_target_oh[b * batch_size: (b+1) * batch_size])).cuda()
        
        net.zero_grad()
        log_pw, log_qw, log_llh = forward_pass_samples(X, y)
        loss = bayes_loss_function(log_pw, log_qw, log_llh, n_batches, batch_size)
        trainlosses.append(loss.data.cpu().numpy())
        loss.backward()
        optimizer.step()

    X = Variable(torch.Tensor(test_data).cuda(), volatile=True)
    pred = net(X, infer=True)
    _, out = torch.max(pred, 1)
    acc = np.count_nonzero(np.squeeze(out.data.cpu().numpy()) == 
                           np.int32(test_target.ravel())) / float(test_data.shape[0])

    print 'Epoch:', ne, 'TrainLoss:', np.mean(trainlosses), 'TestAcc:', acc

Epoch: 0 TrainLoss: 94.87837 TestAcc: 0.594
Epoch: 1 TrainLoss: 20.877987 TestAcc: 0.88


In [ ]:
torch.save(net, 'bayesmlp')

# DropOut MLP

In [ ]:
droput_p = 0.2

net = MLP(input_size, output_size, droput_p)
net.cuda()

batch_size = 100
n_batches = M / float(batch_size)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

n_train_batches = int(train_data.shape[0] / float(batch_size))

In [ ]:
for ne in range(n_epochs):
    trainlosses = []
    for b in range(n_train_batches):
        
        X = Variable(torch.Tensor(train_data[b * batch_size: (b+1) * batch_size])).cuda()
        y = Variable(torch.LongTensor(train_target[b * batch_size: (b+1) * batch_size]).squeeze(1)).cuda()
        
        net.zero_grad()
        output = net(X)
        loss = nn.CrossEntropyLoss()(output,y)
        trainlosses.append(loss.data.cpu().numpy())
        loss.backward()
        optimizer.step()
    
    net.train(False)
    tX = Variable(torch.Tensor(test_data).cuda(), volatile=True)
    pred = net(tX)
    _, out = torch.max(pred, 1)
    acc = np.count_nonzero(np.squeeze(out.data.cpu().numpy()) == 
                           np.int32(test_target.ravel())) / float(test_data.shape[0])
    net.train(True)
    print 'Epoch:', ne, 'TrainLoss:', np.mean(trainlosses), 'TestAcc:', acc

In [ ]:
torch.save(net, 'plainmlp')